In [18]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy 
from cartopy.util import add_cyclic_point
import cartopy.mpl.ticker as cticker
import xesmf as xe
import matplotlib.colors as colors
import pandas as pd


## Load 2000-2016 files
Rename `Time` to `time`

In [22]:
path='/home/ubuntu/asr/asr15km.anl.2D.*.mon.nc'
# ds=xr.open_mfdataset(path_daily+files_daily, concat_dim='time',combine='nested')
ds_full=xr.open_mfdataset(path, concat_dim='Time',combine='by_coords')

ds_full=ds_full.rename({'Time':'time'})

# ds_full['time']=dates
ds_full

<xarray.Dataset>
Dimensions:   (lev: 4, time: 204, x: 720, y: 720)
Coordinates:
  * time      (time) datetime64[ns] 2000-01-16T11:00:00 ... 2016-12-16T11:00:00
    XLAT      (y, x) float32 dask.array<chunksize=(720, 720), meta=np.ndarray>
    XLONG     (y, x) float32 dask.array<chunksize=(720, 720), meta=np.ndarray>
  * lev       (lev) float64 1.0 2.0 3.0 4.0
Dimensions without coordinates: x, y
Data variables:
    DateTime  (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    ALBBCK    (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    ALBEDO    (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    ALBSI     (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    ICEDEPTH  (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    ISLTYP    (time, y, x) int32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    IVGTYP    (time, y, x) int32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    LAI       (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    LANDMASK  (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    LU_INDEX  (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    PMSL      (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    PSFC      (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    Q2M       (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    RH2M      (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    SEAICE    (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    SH2O      (time, lev, y, x) float32 dask.array<chunksize=(1, 4, 720, 720), meta=np.ndarray>
    SKINTEMP  (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    SMOIS     (time, lev, y, x) float32 dask.array<chunksize=(1, 4, 720, 720), meta=np.ndarray>
    SNOALB    (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    SNOW      (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    SNOWC     (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    SNOWH     (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    SST       (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    T2M       (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    TSLB      (time, lev, y, x) float32 dask.array<chunksize=(1, 4, 720, 720), meta=np.ndarray>
    U10E      (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    U10M      (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    V10E      (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    V10M      (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    VEGFRA    (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
Attributes:
    CDI:                             Climate Data Interface version 1.5.4 (ht...
    Conventions:                     CF-1.4
    TITLE:                           Arctic System Reanalysis - The Ohio Stat...
    WEST-EAST_GRID_DIMENSION:        721
    SOUTH-NORTH_GRID_DIMENSION:      721
    BOTTOM-TOP_GRID_DIMENSION:       34
    GRIDTYPE:                        C
    DIFF_OPT:                        1
    KM_OPT:                          1
    DAMP_OPT:                        0
    MP_PHYSICS:                      7
    RA_LW_PHYSICS:                   0
    RA_SW_PHYSICS:                   0
    SF_SFCLAY_PHYSICS:               5
    SF_SURFACE_PHYSICS:              2
    BL_PBL_PHYSICS:                  1
    CU_PHYSICS:                      1
    SURFACE_INPUT_SOURCE:            1
    SST_UPDATE:                      0
    GRID_FDDA:                       0
    GFDDA_INTERVAL_M:                0
    GFDDA_END_H: 

In [31]:
# fix datetimes

dates=pd.date_range(start='2000-01-01',end='2016-12-01',freq='MS')
ds_full['time'] = dates

## Calculate selected variables climatology and anomalies

In [32]:
vars = ['ALBEDO', 'T2M', 'SKINTEMP', 'SST']
ds=ds_full[vars]

In [33]:
ds_climo=ds.groupby('time.month').mean(dim='time')

In [41]:
ds_anoms=ds.groupby('time.month')-ds_climo
ds_anoms

<xarray.Dataset>
Dimensions:   (time: 204, x: 720, y: 720)
Coordinates:
  * time      (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2016-12-01
    XLAT      (y, x) float32 dask.array<chunksize=(720, 720), meta=np.ndarray>
    XLONG     (y, x) float32 dask.array<chunksize=(720, 720), meta=np.ndarray>
    month     (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: x, y
Data variables:
    ALBEDO    (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    T2M       (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    SKINTEMP  (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    SST       (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>

## Save results to disk

In [43]:
ds_climo=ds_climo.rename({'XLONG': 'lon', 'XLAT': 'lat'})
ds_climo.to_netcdf('/home/ubuntu/asr15km.anl.2D.clim.nc')

In [46]:
ds_anoms=ds_anoms.rename({'XLONG': 'lon', 'XLAT': 'lat'})
ds_anoms.to_netcdf('/home/ubuntu/asr15km.anl.2D.anoms.nc')

In [47]:
ds_anoms

<xarray.Dataset>
Dimensions:   (time: 204, x: 720, y: 720)
Coordinates:
  * time      (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2016-12-01
    lat       (y, x) float32 dask.array<chunksize=(720, 720), meta=np.ndarray>
    lon       (y, x) float32 dask.array<chunksize=(720, 720), meta=np.ndarray>
    month     (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: x, y
Data variables:
    ALBEDO    (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    T2M       (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    SKINTEMP  (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    SST       (time, y, x) float32 dask.array<chunksize=(1, 720, 720), meta=np.ndarray>